This code authenticates using OAuth 2.0 to generate an access token for interacting with the Microsoft Graph API, allowing the fetching and sending of emails. It also validates if the email was successfully sent and received by the recipient mailbox.

In [ ]:
!pip install msal requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 2.3 MB/s eta 0:00:00


In [1]:
import os
from getpass import getpass
from msal import ConfidentialClientApplication
import requests
import requests
from datetime import datetime, timezone

In [2]:
# Check and set environment variables

def get_env_variable(var_name):
    value = os.getenv(var_name)
    if value is None:
        value = getpass(f"Please enter {var_name}: ")
        os.environ[var_name] = value
        print(f"{var_name} has been set.")
    return value

CLIENT_ID = get_env_variable("CLIENT_ID")
TENANT_ID = get_env_variable("TENANT_ID")
CLIENT_SECRET = get_env_variable("CLIENT_SECRET")

CLIENT_ID has been set.
TENANT_ID has been set.
CLIENT_SECRET has been set.


In [3]:
def get_access_token():
    """Fetch an access token from Microsoft Graph API."""
    app = ConfidentialClientApplication(
        CLIENT_ID,
        authority=f"https://login.microsoftonline.com/{TENANT_ID}",
        client_credential=CLIENT_SECRET,
    )
    token_response = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])
    return token_response.get("access_token")

# Generate Access Token
access_token = get_access_token()
print(f"Access Token: {access_token}")


Access Token: eyJ0eXAiOiJKV1QiLCJub25jZSI6Im91cVl3bERtMGJMdXpoM1dvVlFzZHBKQ3BwOFRCUjE0SzVjTk1DcWRZRm8iLCJhbGciOiJSUzI1NiIsIng1dCI6Inp4ZWcyV09OcFRrd041R21lWWN1VGR0QzZKMCIsImtpZCI6Inp4ZWcyV09OcFRrd041R21lWWN1VGR0QzZKMCJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8zYmIxZjA3OC03M2I2LTQzNzYtOTZlNS00ZDllOTM1NjQzOTYvIiwiaWF0IjoxNzMyMzA3MzI5LCJuYmYiOjE3MzIzMDczMjksImV4cCI6MTczMjMxMTIyOSwiYWlvIjoiazJCZ1lOaDU1ZWFUc2xNU080OFpQRXdRdkxHTUJ3QT0iLCJhcHBfZGlzcGxheW5hbWUiOiJtYWlsLWFzc2lzdGFudCIsImFwcGlkIjoiNWJlNDVlNDQtZjA4NS00YTVlLThiYWQtYWNlNDNkMWJkYWVkIiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvM2JiMWYwNzgtNzNiNi00Mzc2LTk2ZTUtNGQ5ZTkzNTY0Mzk2LyIsImlkdHlwIjoiYXBwIiwib2lkIjoiM2ZkZGM2ZWQtMDE1Zi00NjFlLTkzNGItYjExMzgwMjAxNGY4IiwicmgiOiIxLkFXOEJlUEN4TzdaemRrT1c1VTJlazFaRGxnTUFBQUFBQUFBQXdBQUFBQUFBQUFCd0FRQnZBUS4iLCJyb2xlcyI6WyJNYWlsLlJlYWRXcml0ZSIsIlVzZXItTWFpbC5SZWFkV3JpdGUuQWxsIiwiTWFpbC5SZWFkQmFzaWMuQWxsIiwiTWFpbC5SZWFkIiwiTWFpbC5TZW5kI

In [4]:
def fetch_emails(user_email, access_token):
    """Fetch emails for a specific user."""
    url = f"https://graph.microsoft.com/v1.0/users/{user_email}/messages"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {
        "$top": 10,  # Fetch top 10 emails
        "$select": "subject,from,receivedDateTime"  # Select specific fields
    }
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        emails = response.json().get("value", [])
        for email in emails:
            print(f"Subject: {email['subject']}")
            print(f"From: {email['from']['emailAddress']['address']}")
            print(f"Received: {email['receivedDateTime']}\n")
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Test Fetch Emails
user_email = "user1@damg7245.onmicrosoft.com"  # Replace with your admin account email
fetch_emails(user_email, access_token)


Subject: Test Email
From: team6@damg7245.onmicrosoft.com
Received: 2024-11-22T01:48:38Z

Subject: Test Mail from Different Domain Mail
From: nasika.d@northeastern.edu
Received: 2024-11-22T00:32:37Z

Subject: test (with pdf)
From: team6@damg7245.onmicrosoft.com
Received: 2024-11-22T00:17:39Z

Subject: Mail with image
From: team6@damg7245.onmicrosoft.com
Received: 2024-11-21T21:33:53Z

Subject: Test Email
From: team6@damg7245.onmicrosoft.com
Received: 2024-11-21T20:37:44Z

Subject: Manual mail
From: team6@damg7245.onmicrosoft.com
Received: 2024-11-21T20:13:51Z

Subject: Test Email
From: team6@damg7245.onmicrosoft.com
Received: 2024-11-21T20:03:58Z

Subject: Test Email
From: team6@damg7245.onmicrosoft.com
Received: 2024-11-21T19:24:57Z

Subject: Test Email
From: team6@damg7245.onmicrosoft.com
Received: 2024-11-21T19:16:20Z

Subject: Test Email
From: team6@damg7245.onmicrosoft.com
Received: 2024-11-21T18:03:19Z



In [5]:
def send_email_individual(sender_email, recipient_email, subject, body, access_token):
    """Send an email using Microsoft Graph API."""
    url = f"https://graph.microsoft.com/v1.0/users/{sender_email}/sendMail"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    email_data = {
        "message": {
            "subject": subject,
            "body": {"contentType": "Text", "content": body},
            "toRecipients": [{"emailAddress": {"address": recipient_email}}],
        }
    }
    response = requests.post(url, headers=headers, json=email_data)

    if response.status_code == 202:
        print("Email sent successfully.")
    else:
        print(f"Error: {response.status_code}, {response.text}")


In [6]:
# Test Send Email
send_email_individual(
    sender_email="team6@damg7245.onmicrosoft.com",
    recipient_email="user1@damg7245.onmicrosoft.com",
    subject="Test Email",
    body="This is a test email sent from Microsoft Graph API.",
    access_token=access_token
)

Email sent successfully.


In [7]:
def send_email(sender_email, recipient_email, subject, body, access_token):
    """Send an email using Microsoft Graph API."""
    print("[INFO] Sending email...")
    url = f"https://graph.microsoft.com/v1.0/users/{sender_email}/sendMail"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    email_data = {
        "message": {
            "subject": subject,
            "body": {"contentType": "Text", "content": body},
            "toRecipients": [{"emailAddress": {"address": recipient_email}}],
        }
    }
    response = requests.post(url, headers=headers, json=email_data)

    if response.status_code == 202:
        print("[SUCCESS] Email sent successfully.")
        return True
    else:
        print(f"[ERROR] Failed to send email. Status Code: {response.status_code}, Response: {response.text}")
        return False

In [8]:
def fetch_emails(recipient_email, access_token, search_subject):
    """Fetch emails from recipient's inbox using Microsoft Graph API."""
    print("[INFO] Fetching emails from recipient's mailbox...")
    url = f"https://graph.microsoft.com/v1.0/users/{recipient_email}/messages"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    params = {
        "$orderby": "receivedDateTime desc",
        "$top": 10  # Fetch only the latest 10 emails for efficiency
    }
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        print("[SUCCESS] Emails fetched successfully.")
        messages = response.json().get("value", [])
        return messages
    else:
        print(f"[ERROR] Failed to fetch emails. Status Code: {response.status_code}, Response: {response.text}")
        return []

In [9]:
def verify_email(subject, sent_time, emails):
    """Verify if the email is present in the fetched emails using subject and timestamp."""
    print("[INFO] Verifying the email in the recipient's mailbox...")
    for email in emails:
        email_subject = email.get("subject", "")
        email_received_time = email.get("receivedDateTime", "")

        # Convert email received time to offset-aware datetime
        email_received_dt = datetime.fromisoformat(email_received_time.replace("Z", "+00:00"))

        # Ensure sent_time is also offset-aware (assume UTC if not specified)
        sent_time_dt = datetime.fromisoformat(sent_time).replace(tzinfo=timezone.utc)

        # Compare subject and timestamp (within 60 seconds)
        if email_subject == subject and abs((email_received_dt - sent_time_dt).total_seconds()) <= 60:
            print(f"[SUCCESS] Email verified. Subject: {email_subject}, Received at: {email_received_dt}")
            return True

    print("[ERROR] Email not found in the recipient's mailbox.")
    return False

In [10]:
# Main execution flow
access_token = access_token
sender_email = "team6@damg7245.onmicrosoft.com"
recipient_email = "user1@damg7245.onmicrosoft.com"
subject = "Test Email"
body = "This is a test email sent from Microsoft Graph API."
current_time = datetime.utcnow().isoformat()

C:\Users\Pigeon\AppData\Local\Temp\ipykernel_37224\2255936748.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow().isoformat()


In [11]:
# Step 1: Send email
if send_email(sender_email, recipient_email, subject, body, access_token):
    
    # Step 2: Fetch emails from recipient's mailbox
    emails = fetch_emails(recipient_email, access_token, subject)

    # Step 3: Verify the email with subject and timestamp
    verify_email(subject, current_time, emails)

else:
    print("[ERROR] Unable to proceed with verification as email sending failed.")

[INFO] Sending email...
[SUCCESS] Email sent successfully.
[INFO] Fetching emails from recipient's mailbox...
[SUCCESS] Emails fetched successfully.
[INFO] Verifying the email in the recipient's mailbox...
[SUCCESS] Email verified. Subject: Test Email, Received at: 2024-11-22 20:34:11+00:00
